In [1]:
import os

# 1. Explicitly set JAVA_HOME and PATH inside the notebook
os.environ["JAVA_HOME"] = r"C:\Users\ibrah\AppData\Local\Programs\Eclipse Adoptium\jdk-11.0.26.4-hotspot"
os.environ["PATH"] = os.environ["JAVA_HOME"] + r"\bin;" + os.environ["PATH"]


In [3]:
try:
    import findspark
    findspark.init()
except ImportError:
    !pip install findspark
    import findspark
    findspark.init()

In [4]:
# 3. Now safely create SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CricketAnalytics") \
    .master("local[*]") \
    .getOrCreate()

# 4. Print Spark Version to confirm it's working
print("Spark version:", spark.version)

Spark version: 3.5.5


In [5]:
from pyspark.sql.functions import col, sum, count, when, avg, expr

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType